# Operaciones avanzadas con DataFrames

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

# Leemos los datos y quitamos filas con NA y convertimos a numéricas las columnas inferidas incorrectamente
flightsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("gs://ucmbucket/data/flights-jan-apr-2018.csv")

# Convertimos a enteros y re-categorizamos ArrDelay en una nueva columna ArrDelayCat
# None (< 15 min), Slight(entre 15 y 60 min), Huge (> 60 min)

cleanFlightsDF = flightsDF.withColumn("ArrDelayCat", F.when(F.col("ArrDelay") < 15, "None")\
                                                      .when((F.col("ArrDelay") >= 15) & (F.col("ArrDelay") < 60), "Slight")\
                                                      .otherwise("Huge"))\
                           .cache()

## Hagamos algunas preguntas a los datos para obtener conclusiones

Imaginemos que somos los dueños de una web de viajes que rastrea internet en busca de vuelos en agencias y otras páginas, los compara y recomienda el más adecuado para el aeropuerto. Junto con esta recomendación, querríamos dar también información sobre vuelos fiables y no fiables en lo que respecta a la puntualidad. Esto depende de muchos factores, como el origen y destino, duración del vuelo, hora del día, etc.

### Agrupación y agregaciones

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Cuáles son los vuelos (origen, destino) con mayor retraso medio? ¿Cuántos vuelos existen entre cada par de aeropuertos?</p>
<p><b>PISTA</b>: Tras hacer las agregaciones para cada pareja "Origin", "Dest" (una agregación para el retraso medio y otra para contar), aplica el método sort(F.col("avgDelay").desc()) para ordenar de forma decreciente por la nueva columna del retraso medio.
</div>

In [2]:
from pyspark.sql import functions as F

retrasoMedioDF = flightsDF.groupBy("Origin", "Dest").agg(F.mean(F.col("ArrDelay")).alias("avgDelay"),
                                                         F.count("*").alias("cuantos"))\
                                                    .sort(F.col("avgDelay").desc())
retrasoMedioDF.show()

+------+----+------------------+-------+
|Origin|Dest|          avgDelay|cuantos|
+------+----+------------------+-------+
|   RDM| MFR|            1347.0|      2|
|   MDT| HPN|             798.0|      1|
|   ORD| GTF|             212.0|      1|
|   ICT| DAY|             210.0|      1|
|   ELM| ATL|             169.0|      2|
|   DSM| PIA|             168.0|      1|
|   ERI| ITH|             160.0|      1|
|   YNG| PIE|             141.0|      1|
|   CMH| HOU|             120.0|      1|
|   HRL| DAL|             111.0|      1|
|   PPG| HNL|109.85714285714286|     35|
|   HNL| PPG|105.85714285714286|     35|
|   PIE| YNG|             104.0|      1|
|   AVP| SFB|              93.0|      1|
|   ACY| MSY| 87.45454545454545|     11|
|   CPR| LAS|              85.0|      1|
|   LAS| CPR|              82.0|      1|
|   TTN| BNA|              76.5|     10|
|   MSP| PVD|              74.0|      1|
|   TUL| OKC|              69.0|      1|
+------+----+------------------+-------+
only showing top

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Es el avión un medio de transporte fiable? Mostrar el número de vuelos en cada categoría de retraso.</p>
En lugar de llamar agg(F.count("*")), podemos llamar a la transformación count() sobre el resultado de groupBy(), y creará
automáticamente una columna llamada "count" con los conteos para cada grupo.
<p> Ahora agrupar también por cada aeropuerto de origen, y mostrando una columna distinta por cada tipo de retraso, con el recuento. PISTA: utilizar la función pivot("colName").</p>

In [3]:
cleanFlightsDF.groupBy("ArrDelayCat").count().show()

+-----------+-------+
|ArrDelayCat|  count|
+-----------+-------+
|     Slight| 298234|
|       None|2004727|
|       Huge| 200152|
+-----------+-------+



In [4]:
cleanFlightsDF.groupBy("Origin", "Dest", "ArrDelayCat").count().show()

+------+----+-----------+-----+
|Origin|Dest|ArrDelayCat|count|
+------+----+-----------+-----+
|   BOS| LGB|       Huge|   22|
|   BOS| HOU|       None|  194|
|   BUF| JFK|     Slight|  108|
|   FLL| ORH|       None|   71|
|   BOS| DEN|       Huge|   71|
|   MSO| LAS|       None|   29|
|   BIS| SFB|       None|   28|
|   MSY| PIE|       None|   32|
|   BOI| LAS|       None|  235|
|   EWR| CVG|       None|  641|
|   LEX| SFB|     Slight|   15|
|   ROA| CLT|     Slight|  100|
|   CLT| TLH|       None|  365|
|   CLT| GRR|       None|  187|
|   SFO| RDM|       None|  369|
|   DEN| DFW|       None| 1578|
|   COS| ORD|       Huge|   52|
|   LAN| MSP|       None|  203|
|   MSP| RAP|     Slight|   44|
|   SBN| DTW|     Slight|   72|
+------+----+-----------+-----+
only showing top 20 rows



In [5]:
cleanFlightsDF.groupBy("Origin", "Dest").pivot("ArrDelayCat").count().show()

+------+----+----+----+------+
|Origin|Dest|Huge|None|Slight|
+------+----+----+----+------+
|   DSM| EWR|  10|  94|    14|
|   STS| PHX|   9| 105|    14|
|   PHL| MCO| 162|1291|   273|
|   PBI| DCA|  37| 393|    49|
|   ORD| PDX|  23| 528|    85|
|   MCI| MKE|   8| 172|    18|
|   SJC| LIH|   1|  83|     5|
|   PBG| PGD|   1|  19|     6|
|   BQN| MCO|   9|  85|    25|
|   MDW| MEM|  22| 172|    42|
|   FSD| ATL|   9|  83|     9|
|   IAD| ILM|   2|  34|     8|
|   MCI| IAH|  38| 487|    54|
|   TPA| ACY|   4| 112|     4|
|   SHD| LWB|   2|  25|  null|
|   SMF| BUR|  61| 720|   124|
|   ATL| GSP|  47|1080|   106|
|   DSM| MCO|   1|  30|    10|
|   SNA| PHX|  57| 967|   256|
|   SPI| ORD|  50| 255|    37|
+------+----+----+----+------+
only showing top 20 rows



In [6]:
cleanFlightsDF.groupBy("Origin", "Dest").pivot("ArrDelayCat").agg(
    F.count("*").alias("conteo"),
    F.mean("Distance").alias("avgDist")
).show()

+------+----+-----------+------------+-----------+------------+-------------+--------------+
|Origin|Dest|Huge_conteo|Huge_avgDist|None_conteo|None_avgDist|Slight_conteo|Slight_avgDist|
+------+----+-----------+------------+-----------+------------+-------------+--------------+
|   MCI| MKE|          8|       436.0|        172|       436.0|           18|         436.0|
|   TPA| ACY|          4|       913.0|        112|       913.0|            4|         913.0|
|   ORD| PDX|         23|      1739.0|        528|      1739.0|           85|        1739.0|
|   MDW| MEM|         22|       480.0|        172|       480.0|           42|         480.0|
|   SHD| LWB|          2|        87.0|         25|        87.0|         null|          null|
|   STS| PHX|          9|       699.0|        105|       699.0|           14|         699.0|
|   FSD| ATL|          9|       954.0|         83|       954.0|            9|         954.0|
|   SMF| BUR|         61|       358.0|        720|       358.0|       

<div class="alert alert-block alert-success">
<p><b>PREGUNTA</b>: ¿Hay relación entre el día de la semana y el retraso a la salida o a la llegada?</p>
    <p><b>PISTA</b>: Calcula el retraso medio a la salida y a la llegada para cada día de la semana y ordena por una de ellas descendentemente.</p>
    <p> Ahora haz lo mismo para cada día pero solo con el retraso a la llegada, desagregado por cada aeropuerto de salida, utilizando la función pivot(). </p>
</div>

In [7]:
avgDelaysDF = cleanFlightsDF.groupBy("DayOfWeek").agg(F.mean("DepDelay"), F.mean("ArrDelay"))\
                            .orderBy(F.col("avg(DepDelay)").desc())
avgDelaysDF.show()

+---------+------------------+-------------------+
|DayOfWeek|     avg(DepDelay)|      avg(ArrDelay)|
+---------+------------------+-------------------+
|        1|10.430177708665964|  5.391113068725289|
|        5|10.220785437977693|  5.027363815430113|
|        7| 9.142161259888235| 3.2344449424598207|
|        3|  8.47071347600168| 3.0525338339576717|
|        4|  8.35856546210902| 2.7390527404801026|
|        2| 8.246502522185226| 2.8412409647873806|
|        6| 6.278199328016013|-0.5748593305876211|
+---------+------------------+-------------------+



<div class="alert alert-block alert-info">
<p><b>LA FUNCIÓN PIVOT</b>: Puede ser interesante ver, para cada (Origin, Dest), el retraso promedio por
día de la semana. Si agrupamos por esas tres variables (Origin, Dest, DayOfWeek), nuestro resultado tendría demasiadas filas para ser fácil de visualizar (7 x 1009 ya que hay 1009 combinaciones de (Origin, DayOfWeek)). En cambio, vamos a crear 7 columnas, una por día de la semana, en nuestro resultado DF. Lo haremos utilizando una de las variables de agrupación (DayOfWeek) como <i> variable pivot</i>. Como esta variable tiene 7 valores distintos, se crearán 7 columnas nuevas. De esta manera, visualizaremos toda la información de cada combinación (Origen, Dest) condensada en una fila con 7 columnas con los 7 retrasos promedio correspondientes a ese (Origen, Dest) en cada día de la semana.
</div>

In [8]:
byOriginDestWithPivot = cleanFlightsDF.groupBy("Origin", "Dest")\
                                 .pivot("DayOfWeek")\
                                 .agg(F.round(F.mean("ArrDelay"), 2))\
                                 .orderBy("Origin", "Dest")

byOriginDestWithPivot.show()

+------+----+------+------+------+-----+------+------+------+
|Origin|Dest|     1|     2|     3|    4|     5|     6|     7|
+------+----+------+------+------+-----+------+------+------+
|   ABE| ATL|  16.0| 17.12|  6.26|-1.38|  4.46|-12.15|  5.85|
|   ABE| CLT|  0.05|  7.92|  5.18| 0.54| -0.36| 16.09| 12.89|
|   ABE| DTW| 42.21| 19.29| 23.02|16.13| 40.39| -4.15|  9.37|
|   ABE| FLL|  null|  null| 33.88| null|  null| 12.29|  null|
|   ABE| MDT|  null|  null|  null| null|  null|  null|  null|
|   ABE| MYR| -4.67|  -4.0|  null| null| 30.75|  null|  null|
|   ABE| ORD| 17.97| 12.52| 33.69|26.06|   9.0|  9.47| 30.21|
|   ABE| PGD| -0.22|  null|  0.73| null| 72.18|  null|  null|
|   ABE| PHL| 10.44|  4.11| -1.44|-3.32|  0.51|  7.93|  3.03|
|   ABE| PIE|  null|  17.8|  null|-3.59|  null|  -3.2|  2.78|
|   ABE| SFB| -0.16| 27.71| 23.63|  2.0| 32.09| 21.46|  1.67|
|   ABI| DFW| 10.65| 16.36| -0.55| 0.38|  4.64| 16.49|  4.61|
|   ABQ| ATL|  2.28|-15.68|-11.15|-11.5| -6.29| -2.92|-14.38|
|   ABQ|

### Operaciones JOIN

Sería útil tener el retraso medio de una ruta junto a cada vuelo, para ver qué vuelos han traído un retraso mayor o menor que el retraso medio para esa ruta.

<div class="alert alert-block alert-success">
    <b> PREGUNTA </b>:
Usa el averageDelayOriginDestDF creado anteriormente, elimina la columna de conteo y luego únerlo con cleanFlightsDF, utilizando Origin y Dest como columnas de enlace. Finalmente, selecciona solo las columnas Origin, Dest, DayOfWeek, ArrDelay y avgDelay del resultado.
</div>

<div class="alert alert-block alert-info">
    <p><b>BONUS (OPCIONAL)</b>: crear una nueva columna <i>belowAverage</i> que tenga valor True si ArrDelay es menor que el avgDelay de esa ruta, y False en caso contrario. No utilizar la función when() sino el operador de comparación directamente entre columnas, la cual devolverá una columna booleana.
</div>

In [9]:
flightsWithAverage = cleanFlightsDF.join(retrasoMedioDF, on = ["Origin", "Dest"])
flightsWithAverage.select("Origin", "Dest", "ArrDelay", "avgDelay").show()

+------+----+--------+------------------+
|Origin|Dest|ArrDelay|          avgDelay|
+------+----+--------+------------------+
|   ATL| GSP|   -12.0|-1.734910277324633|
|   ATL| GSP|    -3.0|-1.734910277324633|
|   ATL| GSP|    -6.0|-1.734910277324633|
|   ATL| GSP|     6.0|-1.734910277324633|
|   ATL| GSP|   -16.0|-1.734910277324633|
|   ATL| GSP|    -2.0|-1.734910277324633|
|   ATL| GSP|     4.0|-1.734910277324633|
|   ATL| GSP|   -14.0|-1.734910277324633|
|   ATL| GSP|   -17.0|-1.734910277324633|
|   ATL| GSP|   -19.0|-1.734910277324633|
|   ATL| GSP|    -9.0|-1.734910277324633|
|   ATL| GSP|    -6.0|-1.734910277324633|
|   ATL| GSP|   -14.0|-1.734910277324633|
|   ATL| GSP|   -14.0|-1.734910277324633|
|   ATL| GSP|   -22.0|-1.734910277324633|
|   ATL| GSP|   -14.0|-1.734910277324633|
|   ATL| GSP|   -19.0|-1.734910277324633|
|   ATL| GSP|   -21.0|-1.734910277324633|
|   ATL| GSP|   -24.0|-1.734910277324633|
|   ATL| GSP|   -20.0|-1.734910277324633|
+------+----+--------+------------

In [23]:
flightsComparedWithAverage = flightsWithAverage.withColumn("belowAverage", F.col("ArrDelay") < F.col("avgDelay"))

flightsComparedWithAverage.select("Origin", "Dest", "ArrDelay", "avgDelay", "belowAverage").show()

+------+----+--------+-------------------+------------+
|Origin|Dest|ArrDelay|           avgDelay|belowAverage|
+------+----+--------+-------------------+------------+
|   IAD| TPA|     -14|-10.310344827586206|        true|
|   IAD| TPA|       2|-10.310344827586206|       false|
|   IND| BWI|      14|0.49382716049382713|       false|
|   IND| BWI|      -6|0.49382716049382713|        true|
|   IND| BWI|      34|0.49382716049382713|       false|
|   IND| JAX|      11|  7.862068965517241|       false|
|   IND| LAS|      57|-2.2413793103448274|       false|
|   IND| LAS|     -18|-2.2413793103448274|        true|
|   IND| MCI|       2| 3.1296296296296298|        true|
|   IND| MCI|     -16| 3.1296296296296298|        true|
|   IND| MCO|       1|-0.5172413793103449|       false|
|   IND| MCO|      80|-0.5172413793103449|       false|
|   IND| MDW|       1| 11.514018691588785|        true|
|   IND| MDW|      10| 11.514018691588785|        true|
|   IND| MDW|      -4| 11.514018691588785|      

**PREGUNTA**: repetir la operación sin utilizar JOIN sino funciones de ventana

In [4]:
from pyspark.sql import Window

w = Window().partitionBy("Origin", "Dest")

# Para obtener el mismo resultado que antes, además de calcular el retraso medio, añadimos también otra columna con 
# el número de vuelos que hay en el dataset (es decir, entre enero y abril de 2018) para esa ruta. 
flightsWithAverageNoJoinDF = cleanFlightsDF.withColumn("avgDelay", F.mean("ArrDelay").over(w))\
                                           .withColumn("cuantos", F.count("*").over(w))\
                                           .withColumn("belowAverage", F.col("ArrDelay") < F.col("avgDelay"))

flightsWithAverageNoJoinDF.select("Origin", "Dest", "ArrDelay", "avgDelay", "belowAverage").show()

+------+----+--------+------------------+------------+
|Origin|Dest|ArrDelay|          avgDelay|belowAverage|
+------+----+--------+------------------+------------+
|   ATL| GSP|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|    -3.0|-1.734910277324633|        true|
|   ATL| GSP|    47.0|-1.734910277324633|       false|
|   ATL| GSP|    null|-1.734910277324633|        null|
|   ATL| GSP|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|    38.0|-1.734910277324633|       false|
|   ATL| GSP|   -13.0|-1.734910277324633|        true|
|   ATL| GSP|   -15.0|-1.734910277324633|        true|
|   ATL| GSP|    10.0|-1.734910277324633|       false|
|   ATL| GSP|    -6.0|-1.734910277324633|        true|
|   ATL| GSP|    -3.0|-1.734910277324633|        true|
|   ATL| GSP|   -14.0|-1.734910277324633|        true|
|   ATL| GSP|    10.0|-1.734910277324633|       false|
|   ATL| GSP|    14.0|-1.734910277324633|       false|
|   ATL| GSP|    -3.0|-1.734910277324633|        true|
|   ATL| G

<div class="alert alert-block alert-success">
<b> PREGUNTA </b>: Vamos a construir otro DF con información sobre los aeropuertos (en una situación real, tendríamos otra tabla en la base de datos como la tabla de la entidad Aeropuerto). Sin embargo, solo tenemos información sobre algunos aeropuertos. Nos gustaría agregar esta información a cleanFlightsDF como nuevas columnas, teniendo en cuenta que queremos que la información del aeropuerto coincida con el aeropuerto de origen de flightsDF. Utilizar la operación de unión adecuada para asegurarse de que no se perderá ninguna de las filas existentes de cleanFlightsDF después de la unión.
</div>

In [6]:
airportsDF = spark.createDataFrame([
    ("JFK", "John F. Kennedy International Airport", 1948),
    ("LIT", "Little Rock National Airport", 1931),
    ("SEA", "Seattle-Tacoma International Airport", 1949),
], ["IATA", "FullName", "Year"])

In [10]:
joinedFlightsDF = cleanFlightsDF.join(airportsDF, airportsDF.IATA == cleanFlightsDF.Origin, "left_outer")

# PREGUNTA: mostrar algunas filas donde FullName no sea null
joinedFlightsDF.where("FullName is not null")\
               .select("Origin", "FullName", "Dest", "Month", "DayofMonth", "DepTime", "ArrTime")\
               .show(10, False) # False para que no se corte la columna si su contenido es muy ancho (ejemplo: FullName)

+------+------------------------------------+----+-----+----------+-------+-------+
|Origin|FullName                            |Dest|Month|DayofMonth|DepTime|ArrTime|
+------+------------------------------------+----+-----+----------+-------+-------+
|SEA   |Seattle-Tacoma International Airport|JFK |1    |1         |2241   |741    |
|SEA   |Seattle-Tacoma International Airport|LGB |1    |1         |1754   |2016   |
|SEA   |Seattle-Tacoma International Airport|BOS |1    |1         |2351   |805    |
|SEA   |Seattle-Tacoma International Airport|BOS |1    |1         |1437   |2302   |
|SEA   |Seattle-Tacoma International Airport|LGB |1    |1         |1208   |1450   |
|SEA   |Seattle-Tacoma International Airport|JFK |1    |2         |2254   |700    |
|SEA   |Seattle-Tacoma International Airport|LGB |1    |2         |1759   |2032   |
|SEA   |Seattle-Tacoma International Airport|BOS |1    |2         |9      |821    |
|SEA   |Seattle-Tacoma International Airport|BOS |1    |2         |1328   |2

## User-defined functions (UDFs)

Vamos a construir un UDF para convertir millas a kilómetros. Ten en cuenta que esto podría hacerse fácilmente multiplicando directamente la columna de millas por 1.6 (y sería mucho más eficiente), ya que Spark permite el producto entre una columna y un número. En todos los casos en los que Spark proporciona funciones integradas para realizar una tarea (como esta), debes usar esas funciones y no una UDF. Las UDF deben emplearse solo cuando no hay otra opción.

La razón es que las funciones integradas de Spark están optimizadas y Catalyst, el optimizador automático de código integrado en Spark, puede optimizarlo aún más. Sin embargo, las UDF son una caja negra para Catalyst y su contenido no se optimizará, y por lo tanto, generalmente son mucho más lentas.

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Primer paso: crear una función de Python que reciba UN número y lo multiplique por 1.6
def milesToKm(miles):
    return miles*1.6

# Vamos a probarla
print(milesToKm(5)) # 5 millas a km: 8 km

# Segundo paso: crear un objeto UDF que envuelva a nuestra función. 
# Hay que especificar el tipo de dato que devuelve nuestra función
udfMilesToKm = F.udf(milesToKm, DoubleType())

# Con esto, Spark será capaz de llamar a nuestra función milesToKm sobre cada uno de los valores de una columna numérica.
# Spark enviará el código de nuestra función a los executors a través de la red, y cada executor la ejecutará sobre las
# particiones (una por una) que estén en ese executor

# Tercer paso: vamos a probar la UDF añadiendo una nueva columna con el resultado de la conversión
flightsWithKm = cleanFlightsDF.withColumn("DistKm", udfMilesToKm(F.col("Distance")))

flightsWithKm.select("Origin", "Dest", "Distance", "DistKM")\
             .distinct()\
             .show(5)

8.0
+------+----+--------+------+
|Origin|Dest|Distance|DistKM|
+------+----+--------+------+
|   PHL| DFW|  1303.0|2084.8|
|   SEA| LAS|   867.0|1387.2|
|   SLC| SAN|   626.0|1001.6|
|   LAX| BWI|  2329.0|3726.4|
|   RDU| ATL|   356.0| 569.6|
+------+----+--------+------+
only showing top 5 rows



<div class="alert alert-block alert-info">
<p><b>BONUS</b>: Crea tu propia UDF que convierta DayOfWeek en una cadena.
Puedes hacerlo creando una función de Python que reciba un número entero y devuelva el día de la semana,
simplemente leyendo desde un vector de cadenas de longitud 7 el valor en la posición indicada por el argumento entero. Para la UDF, recuerda que tu función devuelve un StringType(). Finalmente, prueba tu UDF creando una nueva columna "DayOfWeekString".
</div>

In [12]:
from pyspark.sql.types import StringType

# First step: we create a python function that turns an Integer into the day of the week as string
def dayOfWeekToString(dayInteger):
    # In our data, Monday is 1 but python lists start at index 0
    daysOfWeek = ["", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return daysOfWeek[dayInteger]

# Second step: we wrap our function with a Spark UDF so that it can be called by Spark on every value of an entire column
# This way Spark is able to send our function to the executors, which will eventually run the function on the partitions
# of the data that each executor has
dayOfWeekStringUDF = udf(dayOfWeekToString, StringType())

# Third step: let's try out our UDF by appending a new column that results from transforming (through the UDF) the
# existing column DayOfWeek
flightsWithDayOfWeekStr = cleanFlightsDF.withColumn("DayOfWeekString", dayOfWeekStringUDF(F.col("DayOfWeek")))

flightsWithDayOfWeekStr.select("Origin", "Dest", "DayOfWeek", "DayOfWeekString")\
                       .distinct()\
                       .show()

+------+----+---------+---------------+
|Origin|Dest|DayOfWeek|DayOfWeekString|
+------+----+---------+---------------+
|   SEA| JFK|        2|        Tuesday|
|   DFW| FLL|        7|         Sunday|
|   PHL| SJU|        1|         Monday|
|   GEG| DFW|        6|       Saturday|
|   OAK| PHX|        2|        Tuesday|
|   LAX| DEN|        7|         Sunday|
|   LAX| SEA|        1|         Monday|
|   OAK| PDX|        4|       Thursday|
|   LAS| PDX|        5|         Friday|
|   SLC| PDX|        5|         Friday|
|   SJC| LIH|        6|       Saturday|
|   SJC| EWR|        7|         Sunday|
|   KOA| ANC|        7|         Sunday|
|   SEA| DTW|        1|         Monday|
|   DTW| SEA|        1|         Monday|
|   SEA| BOS|        3|      Wednesday|
|   JFK| LAS|        5|         Friday|
|   ATL| XNA|        5|         Friday|
|   OAK| SLC|        5|         Friday|
|   MSP| ANC|        5|         Friday|
+------+----+---------+---------------+
only showing top 20 rows

